# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей получены из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки найдем возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Составляем первое представление о данных Яндекс.Музыки.




Основной инструмент аналитика — `pandas`. Импортируем эту библиотеку.

In [46]:
import pandas as pd

Прочитаем файл `yandex_music_project.csv` из папки `/datasets` и сохраним его в переменной `df`:

In [47]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Вывод на экран первых 10 строк таблицы:

In [48]:
df.head(10)

userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            Moscow  08:34:34     Monday  
5  Saint-Petersburg  13:09:41     Friday  
6            Moscow  13:00:07  Wednesday  
7            Moscow  20:47:49  Wednesday  
8            Moscow  09:17:40     Friday  
9  Saint-Petersburg  21:20:49  Wednesday

Одной командой получить общую информацию о таблице:

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Из названия непонятно, что за данные хранятся в столбце. 



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков
Выведем на экран названия столбцов:

In [50]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с хорошим стилем:
* несколько слов в названии запишите в «змеином_регистре»,
* все символы сделайте строчными,
* устраните пробелы.

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [51]:
df = df.rename(columns={'  userID':'user_id', 'Track':'track','  City  ':'city','Day':'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [52]:
df = df.rename(columns = {
    '  userID':'user_id',
    'Track':'track',
    '  City  ':'city',
    'Day':'day'})

Проверим результат. Для этого ещё раз сделаем вывод на экран названия столбцов:

In [53]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений
Сначала посчитаем, сколько в таблице пропущенных значений. Для этого достаточно двух методов `pandas`:

In [54]:
df.isnull().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Замените пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создайте список `columns_to_replace`, переберите его элементы циклом `for` и для каждого столбца выполните замену пропущенных значений:

In [55]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [56]:
df.isnull().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты
Посчитаем явные дубликаты в таблице одной командой:

In [57]:
df.duplicated().sum()

3826

Вызовим специальный метод `pandas`, чтобы удалить явные дубликаты:

In [58]:
df = df.drop_duplicates().reset_index(drop=True)

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [60]:
print("Количество пропусков: {}".format(df.duplicated().sum()))

Количество пропусков: 0


Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлечем нужный столбец датафрейма, 
* применим к нему метод сортировки,
* для отсортированного столбца вызовим метод, который вернёт уникальные значения из столбца.

In [61]:
display(df['genre'].sort_values().unique())

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Просмотрим список и найдем неявные дубликаты названия `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [62]:
def replace_wrong_genres(df, wrong_genres):
    df['genre'] = df['genre'].replace(wrong_genres, 'hiphop')
    result = df[df['genre'] == wrong_genres]['genre'].count()
    return result

Вызовем `replace_wrong_genres()` и передадим ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [63]:
replace_wrong_genres(df,('hip', 'hop','hip-hop'))

0

Проверим, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведем отсортированный список уникальных значений столбца `genre`:

In [64]:
display(df['genre'].sort_values().unique())

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге.  
Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для тренировки сначала выполним каждый из расчётов по отдельности. 

Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.



In [65]:
df.groupby('city')['genre'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. Учтем, что в данных есть информация только о прослушиваниях только за эти дни.

In [66]:
df.groupby('day')['genre'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: genre, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Вы видели, как работает группировка по городу и по дням недели. Теперь напишем функцию, которая объединит два эти расчёта.

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применим последовательную фильтрацию с логической индексацией.

Затем посчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную. Вернем эту переменную из функции.

In [67]:
def number_tracks(df, day, city):
    track_list = df[ (df['day'] == day) & (df['city'] == city) ]
    track_list_count = track_list['genre'].count()
    return track_list_count

Вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [68]:
# количество прослушиваний в Москве по понедельникам
number_tracks(df, 'Monday', 'Moscow')

15740

In [69]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks(df, 'Monday', 'Saint-Petersburg')

5614

In [70]:
# количество прослушиваний в Москве по средам
number_tracks(df, 'Wednesday', 'Moscow')

11056

In [71]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

7003

In [72]:
# количество прослушиваний в Москве по пятницам
number_tracks(df, 'Friday', 'Moscow')

15945

In [73]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks(df, 'Friday', 'Saint-Petersburg')

5895

Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [74]:
columns = ['city', 'monday', 'wednesday', 'friday']
data = [['Moscow', 15347, 10865, 15680],
       ['Saint-Petersburg', 5519, 6913, 5802]
]

table = pd.DataFrame(data = data, columns = columns)

table

city  monday  wednesday  friday
0            Moscow   15347      10865   15680
1  Saint-Petersburg    5519       6913    5802

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [75]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'

moscow_general = df[df['city'] == 'Moscow']

In [76]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'

spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [77]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
# 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    Используйте последовательную фильтрацию с помощью логической индексации.
# 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
#    столбцов и посчитать методом count() количество записей для каждого из
#    присутствующих жанров, получившийся Series записать в переменную
#    genre_df_count
# 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
#    в переменную genre_df_sorted
# 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)


def genre_weekday(df,day,time0,time1):
    genre_df = df[(df['day'] == day) & (df['time'] > time0) & (df['time'] < time1)]
    genre_df_sorted = genre_df.sort_values(by = 'genre', ascending = False).head(10)
    return genre_df_sorted


Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [78]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
# объекты, хранящие время, являются строками и сравниваются как строки
# пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

user_id                       track  \
27939  D7FB50DA             Drumming Circle   
55066  A30EB19D       The Frost Is All Over   
59760  59CBCCE5           Dyolmano Dyulbero   
6934   9473651E  Benimle Uçmak İster Misin?   
59592   53B8F7E                    Me trela   
38299   2D0E6AA   The Song of the Butterfly   
53466  384D7E23                       Drums   
38264  FB8A3EA8            Продажная любовь   
6595   46E7BF9B               Canção do Mar   
48069  D02B55C1            One in a Million   

                                                  artist      genre    city  \
27939                                   Professor Trance  worldbeat  Moscow   
55066                                     Global Journey      world  Moscow   
59760                               The Bulgarian Voices      world  Moscow   
6934                                         Yavuz Çetin      world  Moscow   
59592                                        Koza Mostra      world  Moscow   
38299  Estas Tonne Istvan Sky Kek Eg Pablo Arellano &...      world  Moscow   
53466                                       Les Mustangs      world  Moscow   
38264                                   Мурат Тхагалегов      world  Moscow   
6595                                        Dulce Pontes      world  Moscow   
48069                                            Shaudeh      world  Moscow   

           time     day  
27939  09:30:47  Monday  
55066  08:08:36  Monday  
59760  09:53:19  Monday  
6934   08:42:45  Monday  
59592  09:58:46  Monday  
38299  09:45:12  Monday  
53466  08:26:28  Monday  
38264  09:36:11  Monday  
6595   09:30:58  Monday  
48069  09:30:56  Monday

In [79]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

genre_weekday(spb_general, 'Monday','07:00', '11:00')

user_id                        track              artist  genre  \
30615  692AEC35                       Baiana             Emicida  world   
21583   F33FA8C  Pivni Tkach posh edit light        Katya Chilly  world   
4017   4EFA4BA4      Nisti (feat. Fereydoun)          Armin 2afm  world   
41630  E748BD8F        Саморазвитие личности                йога  world   
29139  6AD9DB50            Malaguena Lecuona           Ben Woods  world   
43094  6B62E9CE       The Color of The Night  L' Amore Orchestra  world   
43425  527071A1                  Mere Sagina              Shabaz  world   
56966  BBE0214C              Balagan Balaban        Klezmerfest!  world   
26003  F018BD36                        Aicha              Khaled  world   
44915  2DD633B3                Parole parole             unknown  world   

                   city      time     day  
30615  Saint-Petersburg  09:24:38  Monday  
21583  Saint-Petersburg  08:51:27  Monday  
4017   Saint-Petersburg  09:23:13  Monday  
41630  Saint-Petersburg  09:42:30  Monday  
29139  Saint-Petersburg  09:35:02  Monday  
43094  Saint-Petersburg  08:56:04  Monday  
43425  Saint-Petersburg  09:02:16  Monday  
56966  Saint-Petersburg  08:00:57  Monday  
26003  Saint-Petersburg  09:24:54  Monday  
44915  Saint-Petersburg  08:50:08  Monday

In [80]:
# вызов функции для вечера пятницы в Москве

genre_weekday(moscow_general, 'Friday','17:00', '23:00')

user_id                               track                 artist  \
52485  5DBE85FD                               Полно             Ветер всем   
16628    58A17B                     Bloeubergstrand          Ben E Madison   
27069  B99CF7C3                          Zamin Safe                Zedbazi   
31576  77BE1010                       Kandıramazsın             Genco Ecer   
27054  838AD000                        Baila Rosita              I Soleado   
24473  8ACBE978  Les Champs-Elysées (Waterloo Road)          Vincent Heden   
15254  1DCC8101                             Pajaros   Leonardo Trincabelli   
26879  2D8B2447                  Kykladitiki Sousta  Vaggelis Konitopoulos   
31888  CAAC4375                         Kervansaray       Muhammet İrmikçi   
52503  2D8B2447            I Sousta Pigaine Mprosta       Giannis Kalatzis   

       genre    city      time     day  
52485  world  Moscow  20:06:10  Friday  
16628  world  Moscow  21:59:56  Friday  
27069  world  Moscow  20:07:01  Friday  
31576  world  Moscow  22:00:25  Friday  
27054  world  Moscow  20:36:31  Friday  
24473  world  Moscow  20:21:15  Friday  
15254  world  Moscow  21:22:49  Friday  
26879  world  Moscow  21:25:01  Friday  
31888  world  Moscow  20:22:48  Friday  
52503  world  Moscow  21:05:55  Friday

In [81]:
# вызов функции для вечера пятницы в Петербурге

genre_weekday(spb_general, 'Friday','17:00', '23:00')

user_id                                              track  \
48906  4417FF22                                            E Samba   
1012   5C525738                                                橄榄树   
2379   529A338E                                               Zina   
18330  1D507F3F                                A Cochabamba Me Voy   
49134  F3FB8127                                          Dari Lolo   
41406  6878926B                                          Kolomeika   
32539  3DFD2198                                       Gortoz A Ran   
31393  238848EA                           The Chicken And The Hawk   
52183  7E67C184  Sky Dancer / Kandroma (Tibetan Prayer to Honor...   
49414  578EA43D                                          No Escuro   

                            artist  genre              city      time     day  
48906  The Carnaval Brasilian Band  world  Saint-Petersburg  21:10:56  Friday  
1012                          华夏乐团  world  Saint-Petersburg  20:46:13  Friday  
2379                      Babylone  world  Saint-Petersburg  21:17:27  Friday  
18330                 Daniele Sepe  world  Saint-Petersburg  21:50:57  Friday  
49134                       Aravod  world  Saint-Petersburg  21:48:19  Friday  
41406                    Klezgoyim  world  Saint-Petersburg  21:48:38  Friday  
32539                Denez Prigent  world  Saint-Petersburg  20:12:18  Friday  
31393               DADDY'S REBELS  world  Saint-Petersburg  20:36:58  Friday  
52183                      unknown  world  Saint-Petersburg  20:00:15  Friday  
49414                  Marina Lima  world  Saint-Petersburg  21:59:00  Friday

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраните его в таблице `moscow_genres`.

In [82]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres

moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

Выведем на экран первые десять строк `moscow_genres`:

In [83]:
# просмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим то же и для Петербурга.

Сгруппируем таблицу `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраните в таблице `spb_genres`:

In [84]:
# одной строкой: группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres

spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

Выведите на экран первые десять строк `spb_genres`:

In [85]:
# просмотр первых 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Вы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.